In [1]:
from yahoo_fin.stock_info import get_data
from sqlalchemy import create_engine
from datetime import datetime as dt
import pandas as pd

In [2]:
tickers = pd.DataFrame({
    'Sector' : ['Delivery', 'Delivery', 'Delivery', 'Medical Goods', 'Medical Goods', 'Medical Goods', 'Medical Services', 'Medical Services', 'Medical Services', 'Retail', 'Retail', 'Retail', 'Market', 'Market', 'Market'],
    'Name' : ['FedEx', 'UPS', 'DHL', 'Johnson & Johnson', 'Cardinal Health', '3M', 'Medical Imaging Corp', 'Community Health Systems', 'Teladoc Health', 'Walmart', 'Amazon', 'CVS', 'Dow Jones', 'S & P 500', 'Nasdaq'],
    'Ticker' : ['FDX', 'UPS', 'DPSGY', 'JNJ', 'CAH', 'MMM', 'MEDD', 'CYH', 'TDOC', 'WMT', 'AMZN', 'CVS', '^DJI', '^GSPC', '^IXIC'],
})

tickers

,Sector,Name,Ticker
0,Delivery,FedEx,FDX
1,Delivery,UPS,UPS
2,Delivery,DHL,DPSGY
3,Medical Goods,Johnson & Johnson,JNJ
4,Medical Goods,Cardinal Health,CAH
5,Medical Goods,3M,MMM
6,Medical Services,Medical Imaging Corp,MEDD
7,Medical Services,Community Health Systems,CYH
8,Medical Services,Teladoc Health,TDOC
9,Retail,Walmart,WMT


In [3]:
test_tickers = pd.DataFrame({
    'Sector' : ['Delivery','Medical Goods', 'Medical Services'],
    'Name' : ['FedEx', 'Johnson & Johnson', 'Community Health Systems'],
    'Ticker' : ['FDX', 'JNJ', 'CYH'],
})

test_tickers

,Sector,Name,Ticker
0,Delivery,FedEx,FDX
1,Medical Goods,Johnson & Johnson,JNJ
2,Medical Services,Community Health Systems,CYH


In [4]:
def get_stock_data(tick_name, start_date, end_date):
    df = get_data(ticker=tick_name, start_date=start_date, end_date=end_date)
    df.reset_index(inplace=True)
    df.rename(columns={'index' : 'date'}, inplace=True)
    df['date'] = df['date'].dt.date
    return(df)

In [5]:
def build_stock_df(tickers, start_date, end_date, max_records):

    # get the ticker results for the time period
    for record in tickers.index:
        results = get_stock_data(tickers['Ticker'][record], start_date, end_date)
        results['company'] = tickers['Name'][record]
        results['sector'] = tickers['Sector'][record]
        
        # only keep the last 100 records
        results = results.tail(max_records)

        # if a combined dataset exists append these new results
        try:
            combined_results = combined_results.append(results, ignore_index=True)
            
        # if combined dataset doesn't exist make this dataframe  the combined results df
        except:
            combined_results = results

    return combined_results

prices_df = build_stock_df(tickers=tickers, 
                           start_date='01/01/2020', 
                           end_date = '04/01/2020', 
                           max_records = 100)
prices_df

,date,open,high,low,close,adjclose,volume,ticker,company,sector
0,2020-01-02,152.419998,155.100006,151.600006,155.100006,154.324097,3027600,FDX,FedEx,Delivery
1,2020-01-03,152.500000,153.339996,151.839996,153.179993,152.413681,2216500,FDX,FedEx,Delivery
2,2020-01-06,151.250000,153.550003,150.169998,153.300003,152.533096,2389700,FDX,FedEx,Delivery
3,2020-01-07,153.360001,155.220001,152.339996,154.800003,154.025589,2140100,FDX,FedEx,Delivery
4,2020-01-08,154.570007,157.750000,153.929993,157.130005,156.343933,3053200,FDX,FedEx,Delivery
...,...,...,...,...,...,...,...,...,...,...
925,2020-03-25,7421.359863,7671.209961,7276.399902,7384.299805,7384.299805,4666440000,^IXIC,Nasdaq,Market
926,2020-03-26,7462.209961,7809.830078,7462.209961,7797.540039,7797.540039,3999850000,^IXIC,Nasdaq,Market
927,2020-03-27,7554.250000,7716.240234,7491.140137,7502.379883,7502.379883,3977010000,^IXIC,Nasdaq,Market
928,2020-03-30,7583.459961,7784.350098,7539.970215,7774.149902,7774.149902,3846900000,^IXIC,Nasdaq,Market
